In [5]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import requests
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 12
import seaborn as sns

     ---------------------------------------- 7.2/7.2 MB 18.4 MB/s eta 0:00:00
     ------------------------------------- 965.4/965.4 kB 63.7 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 26.9 MB/s eta 0:00:00
     ---------------------------------------- 103.2/103.2 kB ? eta 0:00:00
     ---------------------------------------- 55.8/55.8 kB ? eta 0:00:00
     -------------------------------------- 293.3/293.3 kB 2.0 MB/s eta 0:00:00


문피아 무료 Top 50, 인기 급상승 50, 유료 베스트 50, 장르별 무료 best 50을 수집해서 데이터 프레임으로

In [6]:
# 무료 Top 200
targetSite = 'https://www.munpia.com/page/best'
driver = webdriver.Chrome() 
driver.get(targetSite)
time.sleep(1)
request = requests.get(targetSite)
html = request.text
soup = BeautifulSoup(html, 'html.parser') # 크롤링 하려는 페이지 띄우기

for i in range(10): # 가장 마지막 까지 스크롤
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(1)
    
request = requests.get(targetSite)    
html = request.text
soup = BeautifulSoup(html, 'html.parser')
#//*[@id="content"]/div[3]/div[1]/a 
#장르=//*[@id="content"]/div[3]/div[1]/a/span[1], 
#제목=//*[@id="content"]/div[3]/div[1]/a/span[2], 
#작가=//*[@id="content"]/div[3]/div[1]/a/span[3]

#//*[@id="content"]/div[3]/div[2]/div[1]/a, 
#//*[@id="content"]/div[3]/div[2]/a/span[1] 

#//*[@id="content"]/div[3]/div[3]/a

# top3의 장르, 제목, 작가 저장할 리스트
Mun_free_Genre = [] # 장르 저장할 리스트
Mun_free_Titles = [] # 제목 저장할 리스트
Mun_free_Author = [] # 작가 저장할 리스트
Mun_free_Ranking = [i+1 for i in range(200) ]
for i in range(3):
    #print(i+1)
    Mun_free_Genre.append(driver.find_element(By.XPATH, f'//*[@id="content"]/div[3]/div[{i+1}]/a/span[1]').text)
    Mun_free_Titles.append(driver.find_element(By.XPATH, f'//*[@id="content"]/div[3]/div[{i+1}]/a/span[2]').text)
    Mun_free_Author.append(driver.find_element(By.XPATH, f'//*[@id="content"]/div[3]/div[{i+1}]/a/span[3]').text)

#print(Mun_free_Genre)
#print(Mun_free_Titles)
#print(Mun_free_Author)
# 1~3위 정상 입력됨

# 나머지 200위까지 위에처럼 입력

#print(driver.find_element(By.XPATH, '//*[@id="BEST-TABLE"]').text)

#//*[@id="BEST-TABLE"]/li[1] # 그냥 태그
#//*[@id="BEST-TABLE"]/li[2] # 4위
#//*[@id="BEST-TABLE"]/li[2]/a/span[1] # 4위 장르
#//*[@id="BEST-TABLE"]/li[2]/a/span[2] # 4위 제목
#//*[@id="BEST-TABLE"]/li[2]/a/span[3] # 4위 작가
#//*[@id="BEST-TABLE"]/li[8] # 10위 순위 - 2
#//*[@id="BEST-TABLE"]/li[8]/a/span[1] # 10위 장르

# 4위부터 200위까지 리스트에 추가 해야 함 마지막 번호는 198임

for i in range(4, 201, 1):
    Mun_free_Genre.append(driver.find_element(By.XPATH, f'//*[@id="BEST-TABLE"]/li[{i-2}]/a/span[1]').text)
    Mun_free_Titles.append(driver.find_element(By.XPATH, f'//*[@id="BEST-TABLE"]/li[{i-2}]/a/span[2]').text)
    Mun_free_Author.append(driver.find_element(By.XPATH, f'//*[@id="BEST-TABLE"]/li[{i-2}]/a/span[3]').text)

#print(Mun_free_Genre)
#print(Mun_free_Titles)
#print(Mun_free_Author)

Munpia_Free = pd.DataFrame({'제목': Mun_free_Titles, '장르': Mun_free_Genre, '작가':Mun_free_Author}, index=Mun_free_Ranking)
Munpia_Free # 문피아 무료 베스트 200

,제목,장르,작가
1,아카식 경제학,"현대판타지, 퓨전",드림보트
2,거물들이 찾는 천재배우,"현대판타지, 드라마",나일함
3,무제한의 세균술사,"현대판타지, 판타지",잔영JY
4,무신연의 : 여포가 효도를 잘함,"퓨전, 대체역사",별꽃라떼
5,종말의 뱀이 되었다,"판타지, 퓨전",비도™
...,...,...,...
196,어쩌다 수선,"퓨전,무협",심씀
197,평범한 변방 시골 남작가의 평범한 환생자,"판타지,퓨전",극동새우
198,알바생이 연출을 너무 잘함!,현대판타지,박달대게
199,환생사기가 전화위복이 되었다.,"판타지,퓨전",眞펜릴


문피아 월간 유료 베스트 200

In [7]:
# 월간 유료 Top 200
targetSite = 'https://www.munpia.com/page/best/section/plsa.monthly'
driver = webdriver.Chrome() 
driver.get(targetSite)
time.sleep(1)
request = requests.get(targetSite)
html = request.text
soup = BeautifulSoup(html, 'html.parser') # 크롤링 하려는 페이지 띄우기

for i in range(10): # 가장 마지막 까지 스크롤
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(1)
    
request = requests.get(targetSite)    
html = request.text
soup = BeautifulSoup(html, 'html.parser')

Mun_pay_Genre = [] # 유료 장르 저장할 리스트
Mun_pay_Titles = [] # 유료 제목 저장할 리스트
Mun_pay_Author = [] # 유료 작가 저장할 리스트
Mun_pay_Ranking = [i+1 for i in range(200) ]
Mun_pay_Gisu = [] # 유료 지수 저장

for i in range(3): # top 3
    #print(i+1)
    Mun_pay_Genre.append(driver.find_element(By.XPATH, f'//*[@id="content"]/div[3]/div[{i+1}]/a/span[1]').text)
    Mun_pay_Titles.append(driver.find_element(By.XPATH, f'//*[@id="content"]/div[3]/div[{i+1}]/a/span[2]').text)
    Mun_pay_Author.append(driver.find_element(By.XPATH, f'//*[@id="content"]/div[3]/div[{i+1}]/a/span[3]').text)
    Mun_pay_Gisu.append(driver.find_element(By.XPATH, f'//*[@id="content"]/div[3]/div[{i+1}]/a/dl/dd').text)
for i in range(4, 201, 1): # top 4 ~ 200
    Mun_pay_Genre.append(driver.find_element(By.XPATH, f'//*[@id="BEST-TABLE"]/li[{i-2}]/a/span[1]').text)
    Mun_pay_Titles.append(driver.find_element(By.XPATH, f'//*[@id="BEST-TABLE"]/li[{i-2}]/a/span[2]').text)
    Mun_pay_Author.append(driver.find_element(By.XPATH, f'//*[@id="BEST-TABLE"]/li[{i-2}]/a/span[3]').text)
    Mun_pay_Gisu.append(driver.find_element(By.XPATH, f'//*[@id="BEST-TABLE"]/li[{i-2}]/a/span[4]/span').text)
    
#print(Mun_pay_Genre)
#print(Mun_pay_Titles)
#print(Mun_pay_Author)
#print(Mun_pay_Gisu)

Munpia_Pay = pd.DataFrame({'제목': Mun_pay_Titles, '장르': Mun_pay_Genre, '작가':Mun_pay_Author, '지수': Mun_pay_Gisu}, index=Mun_free_Ranking)
Munpia_Pay # 문피아 유료 월간 베스트 200 

,제목,장르,작가,지수
1,하남자의 탑 공략법,"현대판타지, 퓨전",꾸찌꾸찌,"6,314,780"
2,용사의 발라드,판타지,도동파,"3,634,340"
3,회귀수선전(回歸修仙傳),"무협, 퓨전",엄청난,"2,965,750"
4,범상한 변호사의 아공간,현대판타지,서칸더브이,"2,635,070"
5,미혼부 AI로 세상을 지배하다,"현대판타지, 퓨전",소보로설,"2,410,030"
...,...,...,...,...
196,내 회귀에 빌런은 없다,"현대판타지,퓨전",1인5역,"262,220"
197,테니스 코트 위의 절대자,"스포츠,일반소설",김군0619,"260,990"
198,무신들의 단톡방에 초대되었다,"무협,퓨전",엉끼,"258,530"
199,검신환생,"무협,판타지",정석연재,"257,630"


데이터 프레임으로 만든 2개의 유료, 무료 자료들 중 장르가 2개인 것을 나누어 따로 데이터를 저장, 지수를 int로 변경 후 저장

In [8]:
Munpia_Free['장르1'] = Munpia_Free.장르.str.split(',').str[0]
Munpia_Free['장르1'] = Munpia_Free['장르1'].str.strip() 
Munpia_Free['장르2'] = Munpia_Free.장르.str.split(',').str[1]
Munpia_Free['장르2'] = Munpia_Free['장르2'].str.strip() 
Munpia_Free.to_csv('./data/Munpia_free.txt', sep = '\t')

Munpia_Pay['장르1'] = Munpia_Pay.장르.str.split(',').str[0]
Munpia_Pay['장르1'] = Munpia_Pay['장르1'].str.strip()
Munpia_Pay['장르2'] = Munpia_Pay.장르.str.split(',').str[1]
Munpia_Pay['장르2'] = Munpia_Pay['장르2'].str.strip()

Munpia_Pay.지수 = Munpia_Pay.지수.str.replace(',', '')
Munpia_Pay.지수 = Munpia_Pay.지수.astype(int)
Munpia_Pay.지수
Munpia_Pay.to_csv('./data/Munpia_pay.txt', sep = '\t')

데이터프레임을 보니 장르 컬럼이 2개이고 나중에 그래프 띄울 때 방해 될 것 같으니 아예 장르로 컬럼을 만들고 해당하는 장르면 true, 아니면 false인 컬럼을 만들겠음

In [12]:
print(Munpia_Pay.장르1.unique())
print(Munpia_Pay.장르1.value_counts())
print(Munpia_Pay.장르2.unique())
print(Munpia_Pay.장르2.value_counts())



['현대판타지' '판타지' '무협' '대체역사' '퓨전' '스포츠']
현대판타지    96
판타지      37
대체역사     31
스포츠      15
무협       14
퓨전        7
Name: 장르1, dtype: int64
['퓨전' nan '판타지' '드라마' '로맨스' '스포츠' '게임' '현대판타지' '전쟁·밀리터리' '대체역사' '일반소설']
퓨전         59
판타지        44
현대판타지      29
드라마        15
전쟁·밀리터리     6
대체역사        6
게임          4
로맨스         2
일반소설        2
스포츠         1
Name: 장르2, dtype: int64


In [20]:
## 새로 만들려는 컬럼 ['현대판타지', '판타지', '무협', '대체역사', '일반소설',
#  '퓨전', '스포츠', '드라마', '로맨스', '게임', '전쟁·밀리터리' ]
#Munpia_Pay['무협'] = Munpia_Pay[Munpia_Pay.장르1 == '무협' | Munpia_Pay.장르2 == '무협']
#print(Munpia_Pay[['장르1', '장르2']].values)
#print(Munpia_Pay[['장르1', '장르2']].values.ravel('K'))
#print(pd.Series(Munpia_Pay[['장르1', '장르2']].values.ravel('K')).dropna())
#print(pd.Series(Munpia_Pay[['장르1', '장르2']].values.ravel('K')).dropna().unique())
all_genres = pd.Series(Munpia_Pay[['장르1', '장르2']].values.ravel('K')).dropna().unique()
#print(all_genres)
for genre in all_genres:
    Munpia_Pay[genre] = (Munpia_Pay['장르1'] == genre) | (Munpia_Pay['장르2'] 햐== genre)
Munpia_Pay.drop(columns=['장르1','장르2'], inplace=True)
Munpia_Pay

,제목,장르,작가,지수,현대판타지,판타지,무협,대체역사,퓨전,스포츠,드라마,로맨스,게임,전쟁·밀리터리,일반소설
1,하남자의 탑 공략법,"현대판타지, 퓨전",꾸찌꾸찌,6314780,True,False,False,False,True,False,False,False,False,False,False
2,용사의 발라드,판타지,도동파,3634340,False,True,False,False,False,False,False,False,False,False,False
3,회귀수선전(回歸修仙傳),"무협, 퓨전",엄청난,2965750,False,False,True,False,True,False,False,False,False,False,False
4,범상한 변호사의 아공간,현대판타지,서칸더브이,2635070,True,False,False,False,False,False,False,False,False,False,False
5,미혼부 AI로 세상을 지배하다,"현대판타지, 퓨전",소보로설,2410030,True,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,내 회귀에 빌런은 없다,"현대판타지,퓨전",1인5역,262220,True,False,False,False,True,False,False,False,False,False,False
197,테니스 코트 위의 절대자,"스포츠,일반소설",김군0619,260990,False,False,False,False,False,True,False,False,False,False,True
198,무신들의 단톡방에 초대되었다,"무협,퓨전",엉끼,258530,False,False,True,False,True,False,False,False,False,False,False
199,검신환생,"무협,판타지",정석연재,257630,False,True,True,False,False,False,False,False,False,False,False
